# Optimized-Pathfinding-for-Safe-Navigation-in-Flooded-Urban-Environments-Using-Grid-Based-Scoring

### 1.	Environment

In [ ]:
# Importing required libraries
import pandas as pd
import numpy as np
import random

In [ ]:
#Code Block : Set Initial State (Must handle dynamic inputs)

#-------------- SET INITIAL STATE --------------
def setInitialState(x,y):
  global initialState
  initialState = (x,y)

In [ ]:
#Code Block : Set the matrix for transition & cost (as relevant for the given problem)

#-------------------- SET ENVIRONMENT MATRIX --------------
# Function to read envronment matrix
def setEnvironmentMatrix(enviM):
  global environmentMatrix
  environmentMatrix = enviM

#----------------- COST FUNCTION -----------------
def computeCostOfAgent(environmentMat, row, col):
  global costofAgent
  """
  Considering that 1 is a water body (-5 points)
                   2 is blockade (-3 points)
                   0 is safe (+5 points)
                  (row, col) are the current coordinates
  """
  # Initializing cost
  cost = 0

  # Check up
  # condition on when to DEDUCT 5 points (Water Body)
  if row > 0 and environmentMat[row - 1][col] == 1:
      cost -= 5
  # condition on when to Deduct 3 points (Flooded Area)
  elif row > 0 and environmentMat[row - 1][col] == 2:
      cost -= 3

  # Check left
  # condition on when to DEDUCT 5 points (Water Body)
  if col > 0 and environmentMat[row][col - 1] == 1:
      cost -= 5
  # condition on when to Deduct 3 points (Flooded Area)
  elif col > 0 and environmentMat[row][col - 1] == 2:
      cost -= 3

  # Check down
  # condition on when to DEDUCT 5 points (Water Body)
  if row < len(environmentMat) - 1 and environmentMat[row + 1][col] == 1:
      cost -= 5
  # condition on when to Deduct 3 points (Flooded Area)
  elif row < len(environmentMat) - 1 and environmentMat[row + 1][col] == 2:
      cost -= 3

  # Check right
  # condition on when to DEDUCT 5 points (Water Body)
  if col < len(environmentMat[0]) - 1 and environmentMat[row][col + 1] == 1:
      cost -= 5
  # condition on when to Deduct 3 points (Flooded Area)
  elif col < len(environmentMat[0]) - 1 and environmentMat[row][col + 1] == 2:
      cost -= 3
  # condition on when to ADD 5 points (Safe Place)
  # All sides are 0
  if (row == 0 or environmentMat[row - 1][col] == 0) and \
      (col == 0 or environmentMat[row][col - 1] == 0) and \
      (row == len(environmentMat) - 1 or environmentMat[row + 1][col] == 0) and \
      (col == len(environmentMat[0]) - 1 or environmentMat[row][col + 1] == 0):
      cost += 5
  # Returns cost
  return cost

In [ ]:
#Code Block : Write function to design the Transition Model/Successor function. Ideally this would be called while search algorithms are implemented

#-------------- TRANSITION MODEL FOR THE AGENT MOVEMENT ----------------
def transitionModel(currentState, action):
  currentX, currentY = currentState
  newX, newY = currentState   # Initialize new coordinates are current only

  # Moving should not go beyond the boundaries and no blockage
  if(action.lower() == 'up' and currentX>0 and environmentMatrix[currentX-1][currentY]!=2):
    newX-=1

  elif(action.lower() == 'down' and currentX<len(environmentMatrix)-1 and environmentMatrix[currentX + 1][currentY] != 2):
    newX+=1

  elif(action.lower() == 'left' and currentY > 0 and environmentMatrix[currentX][currentY - 1] != 2):
    newY-=1

  elif(action.lower() == 'right' and currentY < len(environmentMatrix[0]) - 1 and environmentMatrix[currentX][currentY + 1] != 2):
    newY+=1

  return (newX,newY)

In [ ]:
#Code block : Write fucntion to handle goal test (Must handle dynamic inputs). Ideally this would be called while search algorithms are implemented

#-------------- SET GOAL STATE --------------
def setGoalState(x,y):
  global goalState
  goalState = (x,y)

#-------------- CHECK IF CURRENT STATE IS GOAL STATE --------------
def checkIfCurrentIsGoal(currentState):
  global goalState
  if(currentState == goalState):
    return True
  return False

In [ ]:
# Performance Metrices
# Time and Space Complexity

metrices = ['Time Complexity', 'Space Complexity']

performanceMetrices = {
    'Greedy_Best_First_Search': dict.fromkeys(metrices),
    'Genetic_Algorithm': dict.fromkeys(metrices),
}

### 2.	Definition of Algorithm 1 (Greedy Best First Search)

In [ ]:
# import
import heapq

# HEURISTIC CHOSEN (Manhattan distance)
def computeHeuristic(state, goal):
    return abs(state[0] - goal[0]) + abs(state[1] - goal[1])

def greedyBestFirstSearch(initialState, goalState):

    global GBF_DataFrame, iterationColumn, openListColumn, closedListColumn
    global nodeBeingChecked, passOrFailColumn, currentCostList

    # Open, Close List, iteration
    openList = [(computeHeuristic(initialState, goalState), initialState)]
    closedList = set()
    iteration = 0
    totalNodesExplored= 0
    # Iterate on Open List
    while openList:
        iteration += 1
        iterationColumn.append(iteration)

        # POP Out the Best
        currentCost, currentState = heapq.heappop(openList)
        nodeBeingChecked.append((currentState, currentCost))
        currentCostList.append(currentCost)
        totalNodesExplored +=1

        if checkIfCurrentIsGoal(currentState):  # Goal reached
            passOrFailColumn.append(True)
            openListColumn.append(openList.copy())
            closedListColumn.append(closedList.copy())
            GBF_DataFrame = pd.DataFrame({
                'ITERATION': iterationColumn,
                'OPEN LIST': openListColumn,
                'CLOSED LIST': closedListColumn,
                'NODE BEING CHECKED': nodeBeingChecked,
                'PASS/FAIL': passOrFailColumn,
                'CURRENT COST' : currentCostList
            })
            #performanceMetrices['Greedy_Best_First_Search']['Space Complexity'] = len(iterationColumn) - 1 # nodes expanded
            return currentState
        else:
            passOrFailColumn.append(False)

        closedList.add(currentState)

        for action in ['up', 'down', 'left', 'right']:
            nextState = transitionModel(currentState, action)
            if nextState not in closedList:
                cost = computeHeuristic(nextState, goalState)
                heapq.heappush(openList, (cost, nextState))

        # Append a copy of the openList and closedList at each iteration
        openListColumn.append(openList.copy())
        closedListColumn.append(closedList.copy())
    performanceMetrices['Greedy_Best_First_Search']['Space Complexity'] = totalNodesExplored


    return None

### 3.	Definition of Algorithm 2 (Genetic algorithm)

In [ ]:
#Function to get obstacles and empty blocks
def get_obstacle_blocks(matrix):
  # Initializing empty lists for obstacles and empty blocks
  blockades = []
  empty_cells = []
  # Defining rows and columns
  nr, nc = len(matrix), len(matrix[0])
  # For each row and column
  for row in range(0, nr):
    for col in range(0, nc):
      # Check if value in the matrix is 1 or2 append as blockage
      if matrix[row][col] == 1 or matrix[row][col] == 2:
        blockades.append([row, col])
      # value in the matrix is 0 append as empty blockage
      else:
        empty_cells.append([row, col])
  return blockades, empty_cells

def generate_chromosome_population(matrix, start, destination, num_chromosomes, blockades):
  # get few chromosomes with vaild paths (with no obstacle as part of path)
  # getting the length of rows and columns
  nr, nc = len(matrix), len(matrix[0])
  # Defining variables
  visited = set()
  feasible_paths = []
  path = []
  # Defining directions
  directions = ((0, 1), (1, 0), (0, -1), (-1, 0))
  def find_feasible_paths(row, col):
    # find feasible paths(equal to num of chromosomes) using depth first search
    visited.add((row, col))
    path.append(list([row, col]))
    # print("path:", path)
    # the block row and col are same as that of destination then append a copy of path to feasible paths.
    if row == destination[0] and col == destination[1]:
      feasible_paths.append(path.copy())
      if len(feasible_paths) ==  num_chromosomes:
        print(feasible_paths)
        return
    else:
      # explore all 4 directions
      for direction in directions:
        new_row = row + direction[0]
        new_col = col + direction[1]
        if new_row < 0 or new_row >= (nr) or new_col < 0 or new_col >= (nc) or (new_row, new_col) in visited:
          # print("new row col: ", new_row, new_col)
          continue
        elif [new_row, new_col] in blockades:
          # print("blockade: ", new_row, new_col)
          continue
        else:
          find_feasible_paths(new_row, new_col)
    path.pop()
    visited.remove((row, col))
  # Calling feasible path
  find_feasible_paths(start[0], start[1])
  print(feasible_paths)
  return feasible_paths

def calculate_fitness_value(matrix, path):
  # calculate fitness value for a chromosome
  nr, nc = len(matrix), len(matrix[0])
  directions = ((0,1), (1,0), (0,-1), (-1,0))
  count_water_body = 0
  count_flooded_road = 0
  count_safe_place = 0
  for block_r, block_c in path:
    safe_block = True
    # check all 4 directions
    for direction in directions:
      neighbour_r = block_r + direction[0]
      neighbour_c = block_c + direction[1]
      if neighbour_r < 0 or neighbour_r >= nr or neighbour_c < 0 or neighbour_c >= nc:
        continue
      if matrix[neighbour_r][neighbour_c] == 1:
        count_water_body += 1
        safe_block = False
      if matrix[neighbour_r][neighbour_c] == 2:
        count_flooded_road += 1
        safe_block = False
    if safe_block:
      count_safe_place += 1
  # function 1: determine safety value of path
  function_safe_value = 5 * count_safe_place - 5 * count_water_body - 3 * count_flooded_road
  # function 2: find path length
  function_path_length = len(path)
  # calculate the fitness value using combination of above two functions
  fitness_value = function_safe_value - function_path_length
  return fitness_value


def selection_step(matrix, chromosome_population):
  num_chromosomes = len(chromosome_population)
  selected_chromosomes = []
  # calculate fitness values for the selected paths
  fitness_values = []
  for path in chromosome_population:
    fitness_values.append(calculate_fitness_value(matrix, path))
  total_fitness_value = sum(fitness_values)
  # calculate probability values
  probability_values = []
  for fitness_value in fitness_values:
    probability_values.append(fitness_value/total_fitness_value)
  # calculate cumulative probabilities\
  cumulative_probability_values = [0]
  for probability_value in probability_values:
    cumulative_probability_values.append(probability_value + cumulative_probability_values[-1])
  cumulative_probability_values.pop(0)
  # spin the roulette to select chromosomes
  print("select chromosomes:")
  print("--------------------")
  for _ in range(num_chromosomes):
    roulette = random.random()
    print("roulette: ", roulette)
    for i in range(len(cumulative_probability_values)):
      if roulette <= cumulative_probability_values[i]:
        # if i>0 and roulette > cumulative_probability_values[i-1]:
        selected_chromosomes.append(chromosome_population[i])
        print("selected index: ", i)
        break
  return selected_chromosomes

def crossover_step(selected_chromosomes):
  num_chromosomes = len(selected_chromosomes)
  def perform_crossover(chromosome_1, chromosome_2, crossover_block_1, crossover_block_2):
    # iterate both the chromosomes to find the position of common blocks i.e. crossoverblock 1 and crossoverblock 2
    # swap the paths of two chromosomes which lie between the two common blocks
    for i_1, path in enumerate(chromosome_1):
      if path[0]==crossover_block_1[0] and path[1]==crossover_block_1[1]:
        break
    for i_2, path in enumerate(chromosome_1):
      if path[0]==crossover_block_2[0] and path[1]==crossover_block_2[1]:
        break
    for j_1, path in enumerate(chromosome_2):
      if path[0]==crossover_block_1[0] and path[1]==crossover_block_1[1]:
        break
    for j_2, path in enumerate(chromosome_2):
      if path[0]==crossover_block_2[0] and path[1]==crossover_block_2[1]:
        break
    cross_chromosome_1 = chromosome_1[:i_1+1] + chromosome_2[j_1+1:j_2+1] + chromosome_1[i_2+1:]
    cross_chromosome_2 = chromosome_2[:j_1+1] + chromosome_1[i_1+1:i_2+1] + chromosome_2[j_2+1:]
    return cross_chromosome_1, cross_chromosome_2

  for i in range(num_chromosomes):
    # take pair of chromosomes from selected chromosomes
    # pick a random block from first half of chromosome 2 and a random block from second half of chromosome 2
    # check if both the blocks from above step exists in chromosome 1
    # if yes, call the perform_crossover function to swap the portion between the two blocks
    # else skip the crossover step
    if i % 2 == 0:
      chromosome_1 = selected_chromosomes[i]
      chromosome_2 = selected_chromosomes[i+1]
      l, r = 0, len(chromosome_2)-1
      mid = (l+r)//2
      j = random.randint(0, mid)
      k = random.randint(mid+1, r)
      if chromosome_2[j] in chromosome_1 and chromosome_2[k] in chromosome_1:
        selected_chromosomes[i], selected_chromosomes[i+1] = perform_crossover(chromosome_1, chromosome_2, chromosome_2[j], chromosome_2[k])
  return selected_chromosomes

def mutation_step(matrix, crossover_chromosomes, blockades, destination):
  # for each chromosome after crossover step, perform the following:
  #   1. select a random block in chromosome path except the start and destination block
  #   2. from the block selected above take a step to right and down
  #   3. from new block generated above, find all feasible paths to destination
  #   4. calculate the fitness value of selected chromosome and all the generated chromosomes from above steps
  #   5. pick the best chromosome based on fitness value and append it to list of mutated chromosomes
  nr, nc = len(matrix), len(matrix[0])
  directions = ((0, 1), (1, 0))
  new_block = [0, 0]
  visited = set()
  path = []
  # feasible_paths = []

  def find_path_to_destination(row, col):
    visited.add((row, col))
    path.append([row, col])
    if row == destination[0] and col == destination[1]:
      feasible_paths.append(path.copy())
    else:
      for direction in directions:
        new_row = row + direction[0]
        new_col = col + direction[1]
        if new_row < 0 or new_row >= nr or new_col < 0 or new_col >= nc or (new_row, new_col) in visited:
          continue
        elif [new_row, new_col] in blockades:
          continue
        else:
          find_path_to_destination(new_row, new_col)
    path.pop()
    visited.remove((row, col))

  mutated_chromosomes = []
  for i, chromosome in enumerate(crossover_chromosomes):
    feasible_paths = []
    chromosome_len = len(chromosome)
    # select a random block for mutation
    j = random.randint(0, chromosome_len-1)
    new_chromosomes = []
    # chromosome_blocks = set(chromosome[j+1:])
    # step to the two directions: right and down
    # and generate all possible paths to destination
    # add these paths to new chromosomes
    for direction in directions:
      new_block_r = chromosome[j][0] + direction[0]
      new_block_c = chromosome[j][1] + direction[1]
      if new_block_r < 0 or new_block_r >= nr or new_block_c < 0 or new_block_c >= nc or [new_block_r, new_block_c] in blockades:
        continue
      find_path_to_destination(new_block_r, new_block_c)
      for feasible_path in feasible_paths:
        new_chromosomes.append(chromosome[0: j+1] + feasible_path)

    # calculate the fitness value of selected chromosome and all the generated chromosomes from above steps
    # pick the best chromosome based on fitness value and add it to list of mutated chromosomes
    best_fitness_value = calculate_fitness_value(matrix, chromosome)
    best_chromosome =  chromosome
    for new_chromosome in new_chromosomes:
      new_chromosome_fitness = calculate_fitness_value(matrix, new_chromosome)
      if new_chromosome_fitness > best_fitness_value:
        best_fitness_value = new_chromosome_fitness
        best_chromosome = new_chromosome

    mutated_chromosomes.append(best_chromosome)
    # print("best_chromosome: ", best_chromosome, len(mutated_chromosomes))
  return mutated_chromosomes

def print_chromosomes(chromosomes, text):
  print("\n", text)
  print("--------------------")
  for i, chromosome in enumerate(chromosomes):
    print(f"Chromosome {i}: {chromosome}")

def genetic_algorithm(matrix, start, destination):
  def iteration_genetic_algo(matrix, start, destination, blockades, chromosome_population):
    selected_chromosomes = selection_step(matrix, chromosome_population)
    print_chromosomes(selected_chromosomes, "selected chromosomes: ")
    crossover_chromosomes = crossover_step(selected_chromosomes)
    print_chromosomes(crossover_chromosomes, "crossover chromosomes: ")
    mutated_chromosomes = mutation_step(matrix, crossover_chromosomes, blockades, destination)
    print_chromosomes(mutated_chromosomes, "mutated chromosomes: ")
    return mutated_chromosomes
  # get blockades and empty cells
  blockades, empty_cells = get_obstacle_blocks(matrix)
  # Defining number of chromosomes
  num_chromosomes = 8
  # generate chromosome population based on number of chromosomes
  chromosome_population = generate_chromosome_population(matrix, start, destination, num_chromosomes, blockades)
  output_chromosomes = chromosome_population
  # run the genetic algorithm based on number of iterations below
  num_iterations = 10
  print_chromosomes(chromosome_population, "population chromosomes: ")
  for iteration in range(num_iterations):
    print(f"\n\niteration: {iteration+1}")
    print("====================================================")
    output_chromosomes = iteration_genetic_algo(matrix, start, destination, blockades, output_chromosomes)

  print("\n\nFinal Result after all iterations: ")
  print("=============================================")
  best_fitness_value = float('-inf')
  for chromosome in output_chromosomes:
    fitness_value = calculate_fitness_value(matrix, chromosome)
    if fitness_value > best_fitness_value:
      best_fitness_value = fitness_value
      best_chromosome = chromosome
    print(f"chromosome: {chromosome}")
    print(f"fitness value: {fitness_value}")

  print(f"best chromosome: {best_chromosome}")
  print(f"best chromosome fitness value: {best_fitness_value}")



### DYNAMIC INPUT

In [ ]:
#Code Block : Function & call to get inputs (start/end state)
initialState = (0,0)
goalState = (0,0)
environmentMatrix = []
costofAgent = 0
# The matrix based on the problem statement (Initial Board configuration)
# 0 - free, 1 - water body, 2 - flood
def take_inputs():
    number_of_rows=int(input('Enter number of rows for the matrix '))
    number_of_column=int(input('Enter number of columns for the matrix '))
    print("Enter the entries in a single line (separated by space): ")
    entries = list(map(int, input().split()))
    # matrix
    matrix = np.array(entries).reshape(number_of_rows, number_of_column)
    start_loc = input('Provide start location (Separated by comma ",") ')
    goal_loc= input('Provide goal location (Separated by comma ",") ')
    if ((int(start_loc.split(',')[0])>=0)& (int(start_loc.split(',')[0])<=number_of_rows)) and ((int(start_loc.split(',')[1])>=0) & (int(start_loc.split(',')[1])<=number_of_column)):
      setInitialState(int(start_loc.split(',')[0]),int(start_loc.split(',')[1]))
    else:
      print("Input invalid")
      take_inputs()
    if int(goal_loc.split(',')[0])>=0 & int(goal_loc.split(',')[0])<=number_of_rows and int(goal_loc.split(',')[1])>=0 & int(goal_loc.split(',')[1])<=number_of_column:
      setGoalState(int(goal_loc.split(',')[0]),int(goal_loc.split(',')[1]))
    else:
      print("Input invalid")
      take_inputs()

option = input('Do you want to provide a matrix (Yes/No) ')
if option == 'Yes':
    take_inputs()
else:
  setEnvironmentMatrix([
                  [0, 0, 0, 0, 1, 1, 0, 0],
                  [0, 2, 2, 0, 0, 0, 0, 2],
                  [0, 0, 0, 0, 0, 0, 0, 0],
                  [2, 2, 0, 1, 0, 0, 0 ,0],
                  [0, 1, 0, 0, 0, 0, 2, 2],
                  [0, 0, 0, 1, 0, 0, 0, 0],
                  [0, 2, 2, 0, 0, 1, 0, 0],
                  [0, 0, 0, 0, 0, 1, 0, 0]
                ])
  setInitialState(0,0)
  setGoalState(7,7)

Do you want to provide a matrix (Yes/No) No


### 4.	Calling the search algorithms

In [ ]:
#Invoke algorithm 1 (Should Print the solution, path, cost etc., (As mentioned in the problem))
# DATAFRAME (FOR STORING EVERYTHING IN THE END)

print("Initial:", initialState)
print("Final:", goalState)
print()

GBF_DataFrame = pd.DataFrame(columns=['ITERATION', 'OPEN LIST', 'CLOSED LIST', 'NODE BEING CHECKED', 'PASS/FAIL', 'CURRENT COST'])
iterationColumn = []
openListColumn = []
closedListColumn = []
nodeBeingChecked = []
passOrFailColumn = []
currentCostList = []
result = greedyBestFirstSearch(initialState, goalState)
finalPCost = np.sum(GBF_DataFrame['CURRENT COST'])
performanceMetrices['Greedy_Best_First_Search']['Space Complexity'] = len(GBF_DataFrame['NODE BEING CHECKED'])

if result:
    print(f"Goal reached at {result}")
    print(f"Path Cost for Greedy Best First Search is : {finalPCost}")
else:
    print("No solution found.")

Initial: (0, 0)
Final: (7, 7)

Goal reached at (7, 7)
Path Cost for Greedy Best First Search is : 137


In [ ]:
GBF_DataFrame

,ITERATION,OPEN LIST,CLOSED LIST,NODE BEING CHECKED,PASS/FAIL,CURRENT COST
0,1,"[(13, (0, 1)), (13, (1, 0))]","{(0, 0)}","((0, 0), 14)",False,14
1,2,"[(12, (0, 2)), (13, (1, 0))]","{(0, 1), (0, 0)}","((0, 1), 13)",False,13
2,3,"[(11, (0, 3)), (13, (1, 0))]","{(0, 1), (0, 2), (0, 0)}","((0, 2), 12)",False,12
3,4,"[(10, (0, 4)), (13, (1, 0)), (10, (1, 3))]","{(0, 1), (0, 2), (0, 3), (0, 0)}","((0, 3), 11)",False,11
4,5,"[(9, (0, 5)), (9, (1, 4)), (10, (1, 3)), (13, ...","{(0, 1), (0, 2), (0, 4), (0, 0), (0, 3)}","((0, 4), 10)",False,10
5,6,"[(8, (0, 6)), (8, (1, 5)), (10, (1, 3)), (13, ...","{(0, 1), (0, 2), (0, 4), (0, 5), (0, 0), (0, 3)}","((0, 5), 9)",False,9
6,7,"[(7, (0, 7)), (8, (1, 5)), (7, (1, 6)), (13, (...","{(0, 1), (0, 2), (0, 4), (0, 5), (0, 0), (0, 3...","((0, 6), 8)",False,8
7,8,"[(7, (1, 6)), (8, (1, 5)), (10, (1, 3)), (13, ...","{(0, 1), (0, 7), (0, 4), (0, 0), (0, 3), (0, 6...","((0, 7), 7)",False,7
8,9,"[(6, (2, 6)), (8, (1, 5)), (8, (1, 5)), (13, (...","{(0, 1), (0, 7), (0, 4), (0, 0), (0, 3), (0, 6...","((1, 6), 7)",False,7
9,10,"[(5, (2, 7)), (5, (3, 6)), (7, (2, 5)), (8, (1...","{(0, 1), (0, 7), (0, 4), (0, 0), (0, 3), (0, 6...","((2, 6), 6)",False,6


In [ ]:
#Invoke algorithm 2 (Should Print the solution, path, cost etc., (As mentioned in the problem))
import sys
sys.setrecursionlimit(15500)
genetic_algorithm(environmentMatrix,initialState,goalState)

[[[0, 0], [0, 1], [0, 2], [0, 3], [1, 3], [1, 4], [1, 5], [1, 6], [2, 6], [2, 7], [3, 7], [3, 6], [3, 5], [4, 5], [5, 5], [5, 6], [5, 7], [6, 7], [7, 7]], [[0, 0], [0, 1], [0, 2], [0, 3], [1, 3], [1, 4], [1, 5], [1, 6], [2, 6], [2, 7], [3, 7], [3, 6], [3, 5], [4, 5], [5, 5], [5, 6], [5, 7], [6, 7], [6, 6], [7, 6], [7, 7]], [[0, 0], [0, 1], [0, 2], [0, 3], [1, 3], [1, 4], [1, 5], [1, 6], [2, 6], [2, 7], [3, 7], [3, 6], [3, 5], [4, 5], [5, 5], [5, 6], [6, 6], [6, 7], [7, 7]], [[0, 0], [0, 1], [0, 2], [0, 3], [1, 3], [1, 4], [1, 5], [1, 6], [2, 6], [2, 7], [3, 7], [3, 6], [3, 5], [4, 5], [5, 5], [5, 6], [6, 6], [7, 6], [7, 7]], [[0, 0], [0, 1], [0, 2], [0, 3], [1, 3], [1, 4], [1, 5], [1, 6], [2, 6], [2, 7], [3, 7], [3, 6], [3, 5], [4, 5], [4, 4], [5, 4], [5, 5], [5, 6], [5, 7], [6, 7], [7, 7]], [[0, 0], [0, 1], [0, 2], [0, 3], [1, 3], [1, 4], [1, 5], [1, 6], [2, 6], [2, 7], [3, 7], [3, 6], [3, 5], [4, 5], [4, 4], [5, 4], [5, 5], [5, 6], [5, 7], [6, 7], [6, 6], [7, 6], [7, 7]], [[0, 0], [0

### 5.	Comparitive Analysis

In [ ]:
#Code Block : Print the Time & Space complexity of algorithm 1

import time

start = time.time()
greedyBestFirstSearch(initialState, goalState)
end = time.time()
runtime = end - start

performanceMetrices['Greedy_Best_First_Search']['Time Complexity'] = runtime

print('Greedy Best First Search:')

for k, v in performanceMetrices['Greedy_Best_First_Search'].items():
    print("{:<20} {}".format(k, v))

Greedy Best First Search:
Time Complexity      0.002757549285888672
Space Complexity     21


In [ ]:
#Code Block : Print the Time & Space complexity of algorithm 2

import time

start = time.time()
genetic_algorithm(environmentMatrix, initialState, goalState)
end = time.time()
runtime = end - start

performanceMetrices['Genetic_Algorithm']['Time Complexity'] = runtime
num_chromosomes = 8
num_rows, num_cols = len(environmentMatrix), len(environmentMatrix[0])
performanceMetrices['Genetic_Algorithm']['Space Complexity'] = num_chromosomes * (num_rows + num_cols) * 1.5
# print(f"{num_chromosomes}, {num_rows}, {num_cols} ")
# print(performanceMetrices['Genetic_Algorithm']['Space Complexity'] )

print('Genetic_Algorithm:')

for k, v in performanceMetrices['Genetic_Algorithm'].items():
    print("{:<20} {}".format(k, v))

[[[0, 0], [0, 1], [0, 2], [0, 3], [1, 3], [1, 4], [1, 5], [1, 6], [2, 6], [2, 7], [3, 7], [3, 6], [3, 5], [4, 5], [5, 5], [5, 6], [5, 7], [6, 7], [7, 7]], [[0, 0], [0, 1], [0, 2], [0, 3], [1, 3], [1, 4], [1, 5], [1, 6], [2, 6], [2, 7], [3, 7], [3, 6], [3, 5], [4, 5], [5, 5], [5, 6], [5, 7], [6, 7], [6, 6], [7, 6], [7, 7]], [[0, 0], [0, 1], [0, 2], [0, 3], [1, 3], [1, 4], [1, 5], [1, 6], [2, 6], [2, 7], [3, 7], [3, 6], [3, 5], [4, 5], [5, 5], [5, 6], [6, 6], [6, 7], [7, 7]], [[0, 0], [0, 1], [0, 2], [0, 3], [1, 3], [1, 4], [1, 5], [1, 6], [2, 6], [2, 7], [3, 7], [3, 6], [3, 5], [4, 5], [5, 5], [5, 6], [6, 6], [7, 6], [7, 7]], [[0, 0], [0, 1], [0, 2], [0, 3], [1, 3], [1, 4], [1, 5], [1, 6], [2, 6], [2, 7], [3, 7], [3, 6], [3, 5], [4, 5], [4, 4], [5, 4], [5, 5], [5, 6], [5, 7], [6, 7], [7, 7]], [[0, 0], [0, 1], [0, 2], [0, 3], [1, 3], [1, 4], [1, 5], [1, 6], [2, 6], [2, 7], [3, 7], [3, 6], [3, 5], [4, 5], [4, 4], [5, 4], [5, 5], [5, 6], [5, 7], [6, 7], [6, 6], [7, 6], [7, 7]], [[0, 0], [0

### 6.	Provide your comparitive analysis or findings in no more than 3 lines in below section


Comparison :
 - Greedy Best First Search is comparitively faster than Genetic algorithm in identifying the optimal path.
 - Space complexity of genetic algorithm = O(number of chromosomes * average chromosome path length).
 -- For the default matrix,
     
     
      No. of rows = 8

      No. of columns = 8
     
      Avg. chromosome path length = (no. of rows + no. of columns) * 1.5 = 24
   
      No. of chromosome = 8. Therefore, space complexity = 24*8 = 192

 - Space complexity of greedy best first = Total no. of nodes explored
 -- For the default matrix,

      Total No. of nodes explored = Space Complexity = 21